<a href="https://colab.research.google.com/github/GwiHwan-Go/Detect_SC/blob/main/collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install bs4 pandas
## 자살 모니터링 활동 자료 수집 코드 : DCinside 우울증 갤러리 수집 ##
## 참고 : Kokosei J(https://m.blog.naver.com/ons15/221788312880)
## https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 준비 사항

본 파일을 실행하기에 앞서 
Gdrive에 

__Detect_SC_symptoms__ 이름의

폴더를 생성해 줍니다.

아래 코드는 만들어 준 폴더의 위치로 현재 실행 디렉토리를 설정해주는 코드입니다.

이렇게 디렉터리를 설정해주는 이유는, 수집한 파일을 비교적 쉽게 Gdrive에 저장하기 위해서 입니다.

** 데이터 저장 없이 코드 실행 여부만 확인하고 싶다면, 아래 코드를 건너뛰어도 됩니다.

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Detect_SC_symptoms'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Detect_SC_symptoms


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd 
import requests
from lxml.html import fromstring
import random


In [ ]:
###PARAMS
headers = {"User-Agent": "Monika (Android;)", #User-Agent를 줘서, 사이트의 차단을 피합니다.
           "Cookie": "list_count=100"}
gallid = 'depression_new1' #DC inside의 우울증 갤러리의 website 주소입니다.

# 포맷으로 URL 생성
url = "https://m.dcinside.com/board/{}?page={{}}".format(gallid)
url_desktop = "https://gall.dcinside.com/{{}}board/lists/?id={}&list_num=100&page=1".format(gallid)
url_fortxt = "https://gall.dcinside.com/{{}}board/view/?id={}&no=".format(gallid)
proxy_pool={} # 사이트의 차단을 피하기위한 proxy 집합입니다.

def get_proxies(update=False) :

  # 본 함수는 'https://free-proxy-list.net/' 에서 proxy 를 수집해오는 코드입니다.
  # 만약 위 사이트의 구조가 변경되었다면, 제대로 실행이 되지 않을 수도 있습니다.

  global proxy_pool

  url = 'https://free-proxy-list.net/'
  response = requests.get(url)
  parser = fromstring(response.text)

  if update :
    proxies = update
  else :
    proxies = set()
  for i in parser.xpath('//tbody/tr')[:30]:

    if i.xpath('.//td[7][contains(text(),"yes")]'):
    #Grabbing IP and corresponding PORT
      proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
      proxies.add(proxy)
    
  return proxies

def delete_proxy(li, ele) :
  ## 수집한 proxy 중 인자로 받은 ele 를 지웁니다.
  li.remove(ele)
  return li

def get_end_page_of_site() :

    global url_fortxt
     # 마지막 페이지 번호 파싱하기 위해 데스크탑 페이지 접속
    r = requests.get(url_desktop.format(""), headers = {"User-Agent": "Monika"})
    if "location.replace" in r.text:
        # 마갤이므로 마갤주소로 변환 후 다시 리퀘스트 보냄
        r = requests.get(url_desktop.format("mgallery/"), headers = {"User-Agent": "Monika"})
        # txt파일에 들어갈 url 형식 변환
        url_fortxt = url_fortxt.format("mgallery/")
    else:
        url_fortxt = url_fortxt.format("")
    # 갤 이름, 마지막 페이지 번호 파싱
    soup = bs(r.text, "html.parser")
    # gallname = soup.find("meta", {"name":"title"})['content']
    endpage = int(soup.find("a", {"class":"page_end"})['href'].split("=")[-2].split("&")[0])

    return endpage

def get_page(pagenumb, start, end):

    global proxy_pool, url

    proxy_pool = get_proxies(proxy_pool)
    proxy = random.sample(proxy_pool,1)[0]
    print(pagenumb, "번 페이지 파싱 시작")

    try :

      r = requests.get(url.format(pagenumb), 
                        headers = headers,
                        proxies={"http": proxy}
                        )
      soup = bs(r.text, "html.parser")

      rtnlist = []

      for x in soup.find('ul', {'class': "gall-detail-lst"}).find_all("div", {"class":"gall-detail-lnktb"}):
          # 글제목 추출
          title = x.find('span', {"class":"subjectin"}).text

          # 링크, 글번호 추출
          link = x.find('a')['href']
          id = int(link.split("/")[-1].split("?")[0])

          # 데이터 추출 시 기존 자료와 중복 장지
          if id > start and id < end : 
            print(f"cur_id : {id}, is in the range of {start}~{end}")
            break

          # 작성자 추출
          author = x.find_all("li")[0].text
          date = x.find_all("li")[1].text
          if ":" in date :
            date = f"{localtime.tm_mon}.{localtime.tm_mday}"
          # print(author + " - " + title + ", " + id + " 번 게시글 확인")
          
          rtnlist.append([id, title, author, date, link])
          # print(rtnlist)
      # print(f"Collected {len(rtnlist)} data")

      return rtnlist

    except :
      # 연결이 안될 시 다른 proxy 로 재 시도합니다.
      print("Skipping. Conection errors, try again with another proxy")
      if len(proxy_pool) == 0 :
        print("No available proxy, updating...")
        time.sleep(10)
        proxy_pool = get_proxies(proxy_pool)
      else :
        print("Remove this proxy and try it with another proxy")
        proxy_pool.remove(proxy)
        return []

## 크롤링 데이터 저장 준비

Gdrive 에 

Detect_SC_symptoms 라는 이름의 폴더를 만들어 주었다면

해당 폴더 안에 

data 

라는 폴더를 생성해 줍니다. 수집한 데이터는 앞으로 data 안에 저장될 것입니다.

In [ ]:
# 결과물 저장 경로, 수집한 데이터의 저장 파일 이름.
save_dir = "/content/drive/MyDrive/Detect_SC_symptoms/data"
collected_data_name = "dc_inside_data.csv"

In [ ]:
# 기존 작업한 데이터가 있는 경우에 실행
history_path = save_dir + '/' + collected_data_name
history = pd.read_csv(history_path, low_memory=False, index_col=0) 
first_id, last_id = history.loc[0,'id'], history.loc[history.shape[0]-1,'id']
print("수집한 게시물 범위 :", first_id, "~", last_id, "\n" \
      "수집된 데이터 크기 :", history.shape[0], "\n수집한 데이터 features :", history.columns)
history.tail()

수집한 게시물 범위 : 2 ~ 4439985 
수집된 데이터 크기 : 2498139 
수집한 데이터 features : Index(['id', 'title', 'author', 'date', 'link'], dtype='object')


,id,title,author,date,link
2498134,4439981,일할 때도 같이 붙어 있는단 말이야,_사과_,10.26,https://m.dcinside.com/board/depression_new1/4...
2498135,4439982,이힛 이히힛,노무탱,10.26,https://m.dcinside.com/board/depression_new1/4...
2498136,4439983,뭔데 ㅈㄴ이쁘냐,Maybe,10.26,https://m.dcinside.com/board/depression_new1/4...
2498137,4439984,누가 맨날 내 글에 비추다냐..,노작공원터..,10.26,https://m.dcinside.com/board/depression_new1/4...
2498138,4439985,기분ㄴ씹창났긔윤,제레에반한..,10.26,https://m.dcinside.com/board/depression_new1/4...


## 데이터 수집 시작

본격적으로 데이터 수집을 위한 준비를 마쳤습니다.

아래에서 데이터 수집을 시작합니다.

colab의 특성상 비정상적으로 종료될 경우를 대비해서, 

except 부분에서 현재까지 실행한 파일을 저장하는 코드를 구현하였습니다.

In [ ]:
#MAIN

results = []
localtime = time.localtime()
starttime = time.time()
endpage = get_end_page_of_site()
print("마지막 페이지:", endpage)

print("5 초 후 파싱작업을 시작합니다...")
# time.sleep(5)
try :

  for page in range(100) :
  # for page in range(1,3):
      new_data = get_page(page,first_id,last_id)
      if len(new_data)!=0 :
        results+=new_data
        print(f"Collected {len(results)} data")
  print("파싱 작업 종료. 2 초 후 리스트를 정렬합니다...")
  time.sleep(2)

  print("정렬시작...")
  postlist_sorted = sorted(results, key = lambda x:  int(x[0]))

  # csv로 저장
  df = pd.DataFrame.from_records(postlist_sorted, columns=['id', 'title', 'author', 'link'])
  file_path = save_dir+"/"+f'{localtime.tm_mon}{localtime.tm_mday}{localtime.tm_hour}{localtime.tm_min}.csv'
  df.to_csv(file_path)

  print(f"파일 {file_path}에 저장 완료")
  print("파싱이 완료되었습니다.\n 소요시간: {}초\n프로그램을 종료합니다.".format(time.time() - starttime))

except :

  # 에러났을 때 지금까지 한 거라도 csv로 저장
  postlist_sorted = sorted(results, key = lambda x:  int(x[0]))
  df = pd.DataFrame.from_records(postlist_sorted, columns=['id', 'title', 'author','date', 'link'])
  file_path = save_dir+"/"+f'{localtime.tm_mon}{localtime.tm_mday}{localtime.tm_hour}{localtime.tm_min}.csv'
  df.to_csv(file_path)
  print(f"파일 {file_path}에 저장 완료")
  print("알 지 못할 에러로 프로그램을 종료합니다. \n 소요시간: {}초\n프로그램을 종료합니다.".format(time.time() - starttime))

마지막 페이지: 24874
5 초 후 파싱작업을 시작합니다...
0 번 페이지 파싱 시작
Collected 99 data
1 번 페이지 파싱 시작
Collected 198 data
2 번 페이지 파싱 시작
Collected 298 data
3 번 페이지 파싱 시작
Collected 398 data
4 번 페이지 파싱 시작
Collected 498 data
5 번 페이지 파싱 시작
Collected 598 data
6 번 페이지 파싱 시작
Collected 698 data
7 번 페이지 파싱 시작
Collected 798 data
8 번 페이지 파싱 시작
Collected 898 data
9 번 페이지 파싱 시작
Collected 998 data
10 번 페이지 파싱 시작
Collected 1098 data
11 번 페이지 파싱 시작
Collected 1198 data
12 번 페이지 파싱 시작
Collected 1298 data
13 번 페이지 파싱 시작
Collected 1398 data
14 번 페이지 파싱 시작
Collected 1498 data
15 번 페이지 파싱 시작
Collected 1598 data
16 번 페이지 파싱 시작
Collected 1698 data
17 번 페이지 파싱 시작
Collected 1798 data
18 번 페이지 파싱 시작
Collected 1898 data
19 번 페이지 파싱 시작
Collected 1998 data
20 번 페이지 파싱 시작
Collected 2098 data
21 번 페이지 파싱 시작
Collected 2198 data
22 번 페이지 파싱 시작
Collected 2298 data
23 번 페이지 파싱 시작
Collected 2398 data
24 번 페이지 파싱 시작
Collected 2498 data
25 번 페이지 파싱 시작
Collected 2598 data
26 번 페이지 파싱 시작
Collected 2698 data
27 번 페이지 파싱 시작
Collected 2798 data
28 번 

In [ ]:
## 새로 저장된 파일을 불러와 봅시다. 저장된 파일명은 위 코드의 출력 셀에서 알려주었습니다.
new = pd.read_csv(file_path, low_memory=False, index_col=0) 
final = pd.concat([history,new], axis=0)
final = final.sort_values(by='id', ignore_index=True)

In [ ]:
###ATTENTION : 아래 작업은 데이터를 덮어 씌우는 작업으로, 데이터를 잃어버릴 수도 있으니 신중히 하세요.
print(history.shape, new.shape, final.shape)
print(final.head(5), final.tail(5))

(2498139, 5) (4417, 5) (2502556, 5)
   id         title       author      date  \
0   2    첫글은 내가 먹는다  ㅇㅇ(117.111)  21.03.03   
1   3            ㅎㅇ          ㅠ~ㅠ  21.03.03   
2   5   울갤의 지배자인 나야  ㅇㅇ(117.111)  21.03.03   
3   6       하아잇 사랑해  ㅇㅇ(121.159)  21.03.03   
4   8  ㅅㅂ 내가일빠하려했는데        정병숙녀.  21.03.03   

                                                link  
0  https://m.dcinside.com/board/depression_new1/2...  
1  https://m.dcinside.com/board/depression_new1/3...  
2  https://m.dcinside.com/board/depression_new1/5...  
3  https://m.dcinside.com/board/depression_new1/6...  
4  https://m.dcinside.com/board/depression_new1/8...                 id                title       author   date  \
2502551  4445726   씨발 남자직원이 음식 깔아주니까   연우ㅗ(223.62)  10.27   
2502552  4445727          나도근데아이디두번바꿈          .O2  10.27   
2502553  4445727          나도근데아이디두번바꿈          .O2  10.27   
2502554  4445728  알바 바보 쉨ㅋㅋㅋ 나도 차단해봐라           민애  10.27   
2502555  4445728  알바 바보 쉨ㅋㅋㅋ 나도 차단해봐라           민애  

In [ ]:
### 진짜 덮어 씌우는 거니까, 주의하셔야 합니다.
final_path = "/content/drive/MyDrive/Detect_SC_symptoms/results/dc_inside_data.csv"
final.to_csv(final_path)